In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from scipy import signal

from random import sample, shuffle
from copy import deepcopy

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, GridSearchCV  , StratifiedGroupKFold, cross_validate
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier
from mne.decoding import CSP

import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
dataset_path_root = "/home/bruno/Academico/Doctorado/Neuro_Fairness/Shu_Dataset/000_Dataset_Preprocessing/"
#dataset_path_root = "/home/bzorzet/Academico/Neuro_Fairness/Shu_Dataset/000_Dataset_Preprocessing/"
participants=["sub-001","sub-002","sub-003","sub-004","sub-005",
              "sub-006","sub-007","sub-008","sub-009","sub-010",
              "sub-011","sub-012","sub-013","sub-014","sub-015",
              "sub-016","sub-017","sub-018","sub-019","sub-020",
              "sub-021","sub-022","sub-023","sub-024","sub-025"]
sessions = ["ses-01","ses-02","ses-03","ses-04","ses-05"]

In [3]:
dataset={}
for participant in participants:
    flag_participant=True
    dataset[participant]={}
    for session in sessions: 
        database_path= dataset_path_root + participant +"/"
        data_path=participant+"_"+session+"_task_motorimagery_eeg_preprocessing.mat"
        data=sio.loadmat(database_path + data_path)

        dataset[participant][session]= {}
        dataset[participant][session]['data'] = data['data']
        dataset[participant][session]['labels_trials'] = np.squeeze(data['labels_trials'])
        dataset[participant][session]['n_epochs'] = data['data'].shape[0]
        dataset[participant][session]['n_samples'] = data['data'].shape[2]
        dataset[participant][session]['n_chans'] = data['data'].shape[1]

        if flag_participant and session == sessions[-1]:
            dataset[participant]['group_medidator'] = data['group_medidator']
            dataset[participant]['gender'] = data['gender']
            dataset[participant]['id_participant'] = data['id_participant']
            dataset[participant]['sfreq'] = data['sfreq']
            ch_names = data['ch_names']
            it=0
            for ch in data['ch_names']:
                if ' ' in ch:
                    ch_names[it]=ch[:-1]
                it+=1
            dataset[participant]['ch_names']=ch_names 
            dataset[participant]['age'] = data['age']
            flag_participant = False

In [4]:
dataset[participant].keys()

dict_keys(['ses-01', 'ses-02', 'ses-03', 'ses-04', 'ses-05', 'group_medidator', 'gender', 'id_participant', 'sfreq', 'ch_names', 'age'])

In [5]:
# Concatenamos las sesiones de los sujetos
nchan = dataset[participant][session]['n_chans']
nsamples = dataset[participant][session]['n_samples']

for participant in participants:
    data_=np.zeros((1,nchan,nsamples))
    labels_=np.zeros((1))
    for session in sessions: 
        data_ = np.concatenate((data_, dataset[participant][session]['data']),axis=0)
        labels_ = np.concatenate((labels_, dataset[participant][session]['labels_trials']),axis=0)
    data_ = data_[1:,:,:]
    labels_ = labels_[1:]
    
    dataset[participant]['data_sessions'] = data_.copy()
    dataset[participant]['labels_sessions'] = labels_.copy()
    dataset[participant]['data_gender'] =  np.array(list(dataset[participant]['gender']) * data_.shape[0])
    dataset[participant]['group_participant'] =  np.array(list([participant]) * data_.shape[0])

In [6]:
index_female = []
index_male = []
for participant in participants:
    if dataset[participant]['gender'] == 'M':
        index_male.append(participant)
    elif dataset[participant]['gender'] == 'F':
        index_female.append(participant)
print(f"Participantes hombres: {index_male}")
print(f"Participantes mujeres: {index_female}")

Participantes hombres: ['sub-001', 'sub-002', 'sub-008', 'sub-012', 'sub-013', 'sub-015', 'sub-017', 'sub-018', 'sub-019', 'sub-021', 'sub-022', 'sub-023', 'sub-025']
Participantes mujeres: ['sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-009', 'sub-010', 'sub-011', 'sub-014', 'sub-016', 'sub-020', 'sub-024']


In [7]:
n_features = 6
N_it = 20
n_test_participant = 2
n_val_participant = 2 
n_ign_participant = 1

max_iter = 200
patience = 50 
info_exp = {}
info_clfs = {}
for it in range(N_it):
    dic_aux = {}
    
    X_train = None
    X_val = None
    X_test = None 
    
    X_train_ = None
    X_val_ = None
    X_test_ = None
    
    idx_male = index_male.copy()
    idx_female = index_female.copy()
    # DIVIDE DATASET IN TRAIN, TEST AND VAL
    # TEST PARTICIPANTS:
    idx_male_test = sample(idx_male, n_test_participant)
    idx_female_test = sample(idx_female, n_test_participant)
    for it_ in range(n_test_participant):
        idx_male.remove(idx_male_test[it_])
        idx_female.remove(idx_female_test[it_])
    idx_test = idx_male_test + idx_female_test
    dic_aux['reg_idx_test'] = idx_test   
    # VALIDATION PARTICIPANTS:
    idx_male_val = sample(idx_male, n_val_participant)
    idx_female_val = sample(idx_female, n_val_participant)
    for it_ in range(n_val_participant):
        idx_male.remove(idx_male_val[it_])
        idx_female.remove(idx_female_val[it_])
    idx_val = idx_male_val + idx_female_val
    dic_aux['reg_idx_val'] = idx_val  
    # TRAIN PARTICIPANTS:
    idx_male_ignore = sample(idx_male, n_ign_participant)
    for it_ in range(n_ign_participant):
        idx_male.remove(idx_male_ignore[it_])
    idx_male_train = idx_male.copy()
    idx_female_train = idx_female.copy()
    idx_train = idx_male_train + idx_female_train
    dic_aux['reg_idx_train'] = idx_train


    # CONCATENAMOS EL CONJUNTO DE DATOS
    # TRAIN 
    X_train = np.zeros((1,nchan,nsamples))
    Y_train = np.zeros(1)
    labels_train = np.zeros(1)
    for participant in idx_train:
        X_train = np.concatenate((X_train, dataset[participant]['data_sessions']),axis=0)
        Y_train = np.concatenate((Y_train, dataset[participant]['data_gender']),axis=0)
        labels_train = np.concatenate((labels_train, dataset[participant]['labels_sessions']),axis=0)
    X_train = X_train[1:,:]
    Y_train = Y_train[1:]
    labels_train = labels_train[1:]
    dic_aux['n_trials_train'] = {'male':np.sum(Y_train == 'M'),'female':np.sum(Y_train == 'F')}
    dic_aux['proportion_trials_train'] = {'male':np.sum(Y_train == 'M')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F')),
                                      'female':np.sum(Y_train == 'F')/(np.sum(Y_train == 'M')+np.sum(Y_train == 'F'))}   
    
    # TEST
    X_test = np.zeros((1,nchan,nsamples))
    Y_test = np.zeros(1)
    labels_test = np.zeros(1)
    for participant in idx_test:
        X_test = np.concatenate((X_test, dataset[participant]['data_sessions']),axis=0)
        Y_test = np.concatenate((Y_test, dataset[participant]['data_gender']),axis=0)
        labels_test = np.concatenate((labels_test, dataset[participant]['labels_sessions']),axis=0)
    X_test = X_test[1:,:]
    Y_test = Y_test[1:]
    labels_test = labels_test[1:]
    dic_aux['n_trials_test'] = {'male':np.sum(Y_test == 'M'),'female':np.sum(Y_test == 'F')}
    dic_aux['proportion_trials_test'] = {'male':np.sum(Y_test == 'M')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F')),
                                      'female':np.sum(Y_test == 'F')/(np.sum(Y_test == 'M')+np.sum(Y_test == 'F'))}
    # VALIDATION
    X_val = np.zeros((1,nchan,nsamples))
    Y_val = np.zeros(1)
    labels_val = np.zeros(1)
    for participant in idx_val:
        X_val = np.concatenate((X_val, dataset[participant]['data_sessions']),axis=0)
        Y_val = np.concatenate((Y_val, dataset[participant]['data_gender']),axis=0)
        labels_val = np.concatenate((labels_val, dataset[participant]['labels_sessions']),axis=0)
    X_val = X_val[1:,:]
    Y_val = Y_val[1:]
    labels_val = labels_val[1:]
    dic_aux['n_trials_val'] = {'male':np.sum(Y_val == 'M'),'female':np.sum(Y_val == 'F')}
    dic_aux['proportion_trials_val'] = {'male':np.sum(Y_val == 'M')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F')),
                                      'female':np.sum(Y_val == 'F')/(np.sum(Y_val == 'M')+np.sum(Y_val == 'F'))}
        
    info_exp[f'it_{it}']=dic_aux
    dic_aux={}

    #--------------------------  CSP TRANSFORM -------------------------------#
    csp = CSP(n_components = n_features, reg='oas', norm_trace = True)
    csp.fit(X_train, labels_train)
    X_train_CSP = csp.transform(X_train)
    X_val_CSP = csp.transform(X_val)
    X_test_CSP = csp.transform(X_test)

    #-------------------------CLASSIFIER 1 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = LinearDiscriminantAnalysis(tol=tol)
    
    X_train_CSP_ = scaler.fit_transform(X_train_CSP, Y_train)
    X_val_CSP_ = scaler.transform(X_val_CSP)
    X_test_CSP_ = scaler.transform(X_test_CSP)
    
    
    clf.fit(X_train_CSP_, Y_train)
    acc_train = clf.score(X_train_CSP_ , Y_train)
    acc_val = clf.score(X_val_CSP_ , Y_val)
    acc_test = clf.score(X_test_CSP_, Y_test)
    
    dic_aux['clf_1']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    #-----------------------CLASSIFIER 2 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = svm.SVC(C=1.0, kernel='linear', tol=tol)
    
    X_train_CSP_ = scaler.fit_transform(X_train_CSP, Y_train)
    X_val_CSP_ = scaler.transform(X_val_CSP)
    X_test_CSP_ = scaler.transform(X_test_CSP)
    
    
    clf.fit(X_train_CSP_, Y_train)
    acc_train = clf.score(X_train_CSP_ , Y_train)
    acc_val = clf.score(X_val_CSP_ , Y_val)
    acc_test = clf.score(X_test_CSP_, Y_test)
    
    dic_aux['clf_2']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    #-----------------------CLASSIFIER 3 -----------------------------#
    acc_train = None
    acc_val = None
    acc_test = None
    
    tol = 1e-4
    scaler = StandardScaler()
    clf = svm.SVC(C=1.0, kernel='rbf', tol=tol)
    
    X_train_CSP_ = scaler.fit_transform(X_train_CSP, Y_train)
    X_val_CSP_ = scaler.transform(X_val_CSP)
    X_test_CSP_ = scaler.transform(X_test_CSP)
    
    
    clf.fit(X_train_CSP_, Y_train)
    acc_train = clf.score(X_train_CSP_ , Y_train)
    acc_val = clf.score(X_val_CSP_ , Y_val)
    acc_test = clf.score(X_test_CSP_, Y_test)
    
    dic_aux['clf_3']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test}
    
    
    #-----------------------CLASSIFIER 4 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,8,5), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train_CSP, Y_train)
        X_train_CSP_ = scaler.transform(X_train_CSP)
        X_val_CSP_ = scaler.transform(X_val_CSP)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_CSP_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_CSP_,Y_train))
        acc_val_scores.append(clf.score(X_val_CSP_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_CSP_ = scaler.transform(X_test_CSP)    
    acc_test = best_clf.score(X_test_CSP_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_4']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    #-----------------------CLASSIFIER 5 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,8,3), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train_CSP, Y_train)
        X_train_CSP_ = scaler.transform(X_train_CSP)
        X_val_CSP_ = scaler.transform(X_val_CSP)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_CSP_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_CSP_,Y_train))
        acc_val_scores.append(clf.score(X_val_CSP_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_CSP_ = scaler.transform(X_test_CSP)    
    acc_test = best_clf.score(X_test_CSP_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_5']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    #-----------------------CLASSIFIER 6 -----------------------------#
    acc_train_scores = []
    acc_val_scores = []
    acc_train = 0
    acc_val = 0
    acc_test = 0
    
    # Counter for patience
    it_patience = 0
    
    # Best classifier in early stopping
    best_clf = None
    it_stop = 0 
    
    scaler = StandardScaler()
    clf = MLPClassifier(hidden_layer_sizes=(10,4,2), activation='relu', solver='adam', alpha=0.0001,
                        learning_rate='constant', learning_rate_init=0.01,max_iter=max_iter)
    classes = np.unique(Y_train)

    for it_ in range(max_iter):
        # Scaler fit/transform
        scaler.partial_fit(X_train_CSP, Y_train)
        X_train_CSP_ = scaler.transform(X_train_CSP)
        X_val_CSP_ = scaler.transform(X_val_CSP)
        
        # Classifier fit / evaluate
        clf.partial_fit(X_train_CSP_,Y_train,classes=classes)
        acc_train_scores.append(clf.score(X_train_CSP_,Y_train))
        acc_val_scores.append(clf.score(X_val_CSP_,Y_val))
        
        if acc_val_scores[-1] >= acc_val:
            acc_val = acc_val_scores[-1]
            best_clf = deepcopy(clf)
            it_stop = it_
            it_patience = 0
        else: 
            it_patience += 1
        
        if it_patience >= patience:
            break
        
    X_test_CSP_ = scaler.transform(X_test_CSP)    
    acc_test = best_clf.score(X_test_CSP_,Y_test)
    acc_train = acc_train_scores[it_stop] 
     
    dic_aux['clf_6']={'estimator': clf, 'acc_train': acc_train, 'acc_val' : acc_val, 'acc_test' : acc_test,
                      'acc_train_scores': acc_train_scores, 'acc_val_scores': acc_val_scores, 'it_stop':it_stop}
    
    info_clfs[f'it_{it}']=dic_aux

: 

: 

In [13]:
#Generamos la tabla de los acc para cada iteración y clasificador
n_clfs = 6

matrix_acc_train = np.zeros((n_clfs,N_it))
matrix_acc_val = np.zeros((n_clfs,N_it))
matrix_acc_test = np.zeros((n_clfs,N_it))

for it0 in range(N_it):
    for it1 in range(n_clfs):
        matrix_acc_train[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_train']
        matrix_acc_val[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_val']
        matrix_acc_test[it1,it0] = info_clfs[f'it_{it0}'][f'clf_{it1+1}']['acc_test']

acc_train_clfs_mean = matrix_acc_train.mean(axis=1,keepdims=True)
acc_train_clfs_std = np.std(matrix_acc_train,axis=1,keepdims=True)

acc_val_clfs_mean = matrix_acc_val.mean(axis=1,keepdims=True)
acc_val_clfs_std = np.std(matrix_acc_val,axis=1,keepdims=True)

acc_test_clfs_mean = matrix_acc_test.mean(axis=1,keepdims=True)
acc_test_clfs_std = np.std(matrix_acc_test,axis=1,keepdims=True)

In [19]:
accs_train_df = pd.DataFrame(matrix_acc_train,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_val_df = pd.DataFrame(matrix_acc_val,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_test_df = pd.DataFrame(matrix_acc_test,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
accs_test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
clf_1,0.634677,0.727559,0.630703,0.551724,0.705914,0.566278,0.441053,0.424987,0.514054,0.359125,0.479188,0.661299,0.344271,0.392439,0.488782,0.515894,0.459390,0.323711,0.609704,0.507920
clf_2,0.658537,0.728084,0.646269,0.550663,0.704848,0.564157,0.462632,0.419237,0.518919,0.356990,0.527575,0.695652,0.358333,0.423323,0.506944,0.515373,0.472840,0.309278,0.624798,0.507392
clf_3,0.532874,0.613648,0.534622,0.469496,0.727224,0.430541,0.504737,0.429692,0.522703,0.504803,0.585848,0.601181,0.399479,0.322151,0.586004,0.587806,0.460424,0.423196,0.490027,0.484688
clf_4,0.550371,0.667717,0.590982,0.484881,0.633990,0.402439,0.408421,0.499216,0.586486,0.481857,0.575442,0.621578,0.524479,0.381257,0.626068,0.478895,0.541645,0.463918,0.415633,0.428194
clf_5,0.544539,0.653018,0.447665,0.518833,0.622802,0.464475,0.536842,0.451124,0.544324,0.520811,0.518210,0.593666,0.403125,0.348243,0.570513,0.495571,0.523539,0.436598,0.534232,0.513728
clf_6,0.476140,0.642520,0.639828,0.485411,0.679275,0.461294,0.356842,0.462624,0.607027,0.465848,0.491155,0.546430,0.346354,0.513312,0.615919,0.589891,0.539058,0.495361,0.648518,0.496304


In [21]:
accs_test_df.to_csv('acc_test_prueba_borrar.csv')

In [ ]:
results_df = pd.DataFrame(np.concatenate((acc_train_clfs_mean,acc_train_clfs_std,
                                          acc_val_clfs_mean,acc_val_clfs_std,
                                          acc_test_clfs_mean,acc_test_clfs_std),axis=1),
                          columns = ['TRAIN MEAN', 'TRAIN STD','VAL MEAN', 'VAL STD','TEST MEAN', 'TEST STD']
                          ,index=['clf_1','clf_2','clf_3','clf_4','clf_5','clf_6'])
results_df

,TRAIN MEAN,TRAIN STD,VAL MEAN,VAL STD,TEST MEAN,TEST STD
clf_1,0.642191,0.037987,0.544589,0.127168,0.516933,0.115620
clf_2,0.647471,0.036462,0.544610,0.122844,0.527592,0.118050
clf_3,0.839688,0.019528,0.539080,0.102971,0.510557,0.088911
clf_4,0.771887,0.070144,0.595092,0.106314,0.518174,0.084574
clf_5,0.766542,0.072283,0.606488,0.100791,0.512093,0.070887
clf_6,0.730410,0.083381,0.608424,0.096889,0.527956,0.090642


In [ ]:
from scipy.stats import ks_1samp
from scipy.stats import ttest_1samp
from scipy.stats import wilcoxon
from scipy import stats

In [ ]:
alpha = 0.05
alpha_bonferroni = alpha / 6

test_norm = pd.DataFrame(columns=['statistics','p-value','H0'])
clf_names = accs_test_df.index.values.tolist()

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    norm_test = ks_1samp(acc_test,stats.norm.cdf)
    if norm_test.pvalue <= alpha_bonferroni:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,False]
    else:
        test_norm.loc[clf]=[norm_test.statistic,norm_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test_norm

Significancia: 0.0083
Intervalo de confianza: 99.16666666666667


,statistics,p-value,H0
clf_1,0.626922,3.945285e-08,False
clf_2,0.621445,5.567034e-08,False
clf_3,0.626331,4.095549e-08,False
clf_4,0.648494,9.680973e-09,False
clf_5,0.636171,2.180989e-08,False
clf_6,0.635462,2.283578e-08,False


In [ ]:
mu = 0.5
test = pd.DataFrame(columns=['statics','p-value','H0'])

for clf in clf_names: 
    acc_test = accs_test_df.loc[clf]
    w_test = wilcoxon(acc_test-mu)
    if w_test.pvalue <= alpha_bonferroni:
        test.loc[clf]=[w_test.statistic,w_test.pvalue,False]
    else: 
        test.loc[clf]=[w_test.statistic,w_test.pvalue,True]

print(f'Significancia: {alpha_bonferroni:.2}')
print(f'Intervalo de confianza: {100*(1-alpha_bonferroni)}')
test

Significancia: 0.0083
Intervalo de confianza: 99.16666666666667


,statics,p-value,H0
clf_1,91.0,0.621513,True
clf_2,77.0,0.311794,True
clf_3,90.0,0.595819,True
clf_4,81.0,0.388376,True
clf_5,84.0,0.452375,True
clf_6,77.0,0.311794,True
